# Gyakori elemhalmazok előállítása

## FP-Growth

Az FP-Growth algoritmus a Frequent Pattern Tree (FP-Tree) adatszerkezetet használja, így hatékonyabb a nagyobb adathalmazokon, mivel nem kell előre generálni a kandidátushalmazokat, mint az Apriori esetében.

Telepítsük az [`mlxtend`](https://pypi.org/project/mlxtend/) csomagot.

In [ ]:
!python -m pip install mlxtend --upgrade --user 

Importáljuk a laborhoz szükséges könyvtárakat.

In [ ]:
import pandas as pd  
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
import timeit


Az `fpgrowth` függvény egy one-hot kódolású pandas DataFrame-ben várja az adatokat. Tegyük fel, hogy a következő tranzakciós adatokkal rendelkezünk:

In [2]:
dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

 A következőképpen tudjuk átalakítani az algoritmus számára a `TransactionEncoder` segítségével:

In [ ]:
te = TransactionEncoder()
te_array = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_array, columns=te.columns_)
df

,Apple,Corn,Dill,Eggs,Ice cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,True,True,False,False,False,False
3,False,True,False,False,False,True,True,False,False,True,True
4,False,True,False,True,True,True,False,False,True,False,False


Nézzük meg azokat a gyakori elemhalmazokat, amelyek legalább 60%-os supporttal rendelkeznek:

In [ ]:
fpgrowth(df, min_support=0.6)

,support,itemsets
0,1.0,(5)
1,0.8,(3)
2,0.6,(10)
3,0.6,(6)
4,0.6,(8)
5,0.8,"(3, 5)"
6,0.6,"(10, 5)"
7,0.6,"(5, 6)"
8,0.6,"(8, 3)"
9,0.6,"(8, 5)"


Alapértelmezés szerint az `fpgrowth` függvény visszaadja az elemek oszlopindexeit, ami hasznos lehet a későbbi műveleteknél, például a társítási szabályok bányászatánál. A jobb olvashatóság érdekében beállíthatjuk a `use_colnames=True` értéket, hogy ezeket az egész értékeket a megfelelő elemnevekké alakítsuk:

In [5]:
fpgrowth(df, min_support=0.6, use_colnames=True)

,support,itemsets
0,1.0,(Kidney Beans)
1,0.8,(Eggs)
2,0.6,(Yogurt)
3,0.6,(Milk)
4,0.6,(Onion)
5,0.8,"(Kidney Beans, Eggs)"
6,0.6,"(Yogurt, Kidney Beans)"
7,0.6,"(Kidney Beans, Milk)"
8,0.6,"(Onion, Eggs)"
9,0.6,"(Onion, Kidney Beans)"


### Apriori vs FP-Growth

Mivel az FP-Growth nem igényli kifejezetten a kandidátushalmazok létrehozását, nagyságrendekkel gyorsabb lehet, mint az alternatív Apriori algoritmus. Például a következő cellák összehasonlítják az Apriori algoritmus teljesítményét az FP-Growth teljesítményével. Még ezen a nagyon egyszerű adathalmazon is az FP-Growth gyorsabb.

In [ ]:
te = TransactionEncoder()
te_array = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_array, columns=te.columns_)

In [7]:
timeit.timeit('apriori(df, min_support=0.6)', globals=globals(), number=100)

0.12582207400009793

In [8]:
timeit.timeit('fpgrowth(df, min_support=0.6)', globals=globals(), number=100)

0.10923609700057568

## Feladatok

Mivel már múlt hétről ismerjük a `store_data.csv` tartalmát, töltsük be ismét és próbáljunk ki 2 másik gyakori mintabányászati algoritmust az `mlxtend` könyvtárból: [https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/](https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/). 

Azonos paraméterek mellett mérjük meg mindegyik sebességét.

Foglaljuk össze röviden, hogy milyen eredményere jutottunk. Melyik volt a leggyorsabb algoritmus? Hogyan befolyásolták a futásidőt a megválasztott `min_support` stb. értékek?